In [21]:
!git clone https://github.com/TariqAi/Chatbot-Portfolio.git

Cloning into 'Chatbot-Portfolio'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 24 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (24/24), 100.56 KiB | 1.46 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [8]:
!pip install \
langchain_community \
langchain_pinecone \
langchain_openai \
unstructured \
langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 45.9 MB/s eta 0:00:00
  Preparing metadata

In [9]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import glob

In [22]:
loader = DirectoryLoader('Chatbot-Portfolio', glob="**/*.mdx")

In [11]:
docs = loader.load()

In [12]:
docs[0]

Document(metadata={'source': 'portfolio/src/app/learn/courses/infrastructure-as-code/page.mdx'}, page_content='import { ArticleLayout } from \'@/components/ArticleLayout\' import Image from \'next/image\' import infraAsCode from \'@/images/infrastructure-as-code.webp\' import { Button } from \'@/components/Button\'\n\nimport { createMetadata } from \'@/utils/createMetadata\'\n\nexport const metadata = createMetadata({ author: \'Zachary Proser\', date: \'2023-10-18\', title: \'Infrastructure as Code\', description: \'Learn to stand up infrastructure in AWS using infrastructure as code\', image: infraAsCode, })\n\nexport default (props) => <ArticleLayout metadata={metadata} {...props} />\n\n## Build systems in the cloud - quickly\n\n<Image src={infraAsCode} alt="Infrastructure as Code" />\n\nInfrastructure as code is a critical skill these days. Practitioners are able to define and bring up reproducible copies of architectures on cloud providers such as AWS and Google Cloud.\n\nThis cour

In [13]:
docs[0].metadata

{'source': 'portfolio/src/app/learn/courses/infrastructure-as-code/page.mdx'}

In [14]:
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [15]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

index_name = "pinecone-chatbot"

# Split our documents into chunks
text_splitter = RecursiveCharacterTextSplitter()
split_docs = text_splitter.split_documents(docs)

In [16]:
split_docs[25]

Document(metadata={'source': 'portfolio/src/app/learn/courses/your-first-full-stack-app/page.mdx'}, page_content='import { ArticleLayout } from \'@/components/ArticleLayout\' import yourFirstFullStackApp from \'@/images/first-full-stack-app.webp\' import { Button } from \'@/components/Button\' import Image from \'next/image\';\n\nimport { createMetadata } from \'@/utils/createMetadata\'\n\nexport const metadata = createMetadata({ author: \'Zachary Proser\', date: \'2023-10-18\', title: \'Your first full stack app\', description: \'Frontend, Backend, and Database - soon you&apos;ll have mastery of them all\', image: yourFirstFullStackApp, })\n\nexport default (props) => <ArticleLayout metadata={metadata} {...props} />\n\n<Image width={1200} height={600} src={yourFirstFullStackApp.src} />\n\n## Get your first full stack app on your portfolio\n\nAlways wanted to build a complete web app but aren\'t sure how to get started? This course is for you.\n\n<Button variant={"green"} href={`/learn

In [17]:
vectorstore = PineconeVectorStore.from_documents(split_docs, embeddings, index_name=index_name)

In [18]:
query = "What is the programming bug?"

similar_docs = vectorstore.similarity_search(query)

In [19]:
similar_docs

[Document(metadata={'source': 'portfolio/src/app/blog/first-see-if-youve-got-the-bug/page.mdx'}, page_content='import { ArticleLayout } from \'@/components/ArticleLayout\' import Newsletter from \'@/components/NewsletterWrapper\'\n\nimport sparkOfLearning from \'@/images/spark\n\nof\n\nlearning.webp\'\n\nimport foundTheSpark from \'@/images/found\n\nthe\n\nspark.webp\'\n\nimport programmingBug from \'@/images/programming\n\nbug.webp\'\n\nimport exploringProgramming from \'@/images/exploring\n\nprogramming.webp\'\n\nimport learningToCodeMap from \'@/images/learning\n\nto\n\ncode\n\nmap.webp\'\n\nimport workingHacker from \'@/images/working\n\nhacker.webp\'\n\nimport hookedOnHacking from \'@/images/hooked\n\non\n\nhacking.webp\'\n\nimport hackerEncouragement from \'@/images/hacker\n\nencouraging.webp\'\n\nimport programmingBugReleased from \'@/images/programming\n\nbug\n\nreleased.webp\'\n\nimport programmersInSnowglobe from \'@/images/programmers\n\nin\n\nsnowglobe.webp\'\n\nimport prog

In [20]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

qa.invoke(query)

{'query': 'What is the programming bug?',
 'result': '"The programming bug" refers to the spark of innate curiosity that drives someone\'s learning forward in the field of software development, coding, systems engineering, machine learning, data science, and other computer-related areas. It is the intrinsic motivation that comes from within the individual, pushing them to explore and learn without external pressure.'}

# **Regards, Tariq Ibrahim**